In [ ]:
!pip install beautifulsoup4
!pip install -q bitsandbytes torch accelerate transformers
!pip install langchain faiss-cpu sentence-transformers langchainhub 

In [2]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 3.6 MB/s eta 0:00:0000:0100:01


In [3]:
import requests
import warnings
warnings.simplefilter('ignore')
import re
from datetime import date
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM , AutoTokenizer , BitsAndBytesConfig , pipeline ,VitsModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import JSONLoader
import torch
import json

2024-02-20 13:30:07.591924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 13:30:07.592032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 13:30:07.753208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
home_page_url = "https://ticka.tech/shop/"
sub_pages_url="https://ticka.tech/shop/page/"
home_page = requests.get(home_page_url)
sub_page = requests.get(home_page_url)
soup = BeautifulSoup(home_page.content, "html.parser")

In [6]:
results = soup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")

In [7]:
def preprocessProductName(productName):
  productName= productName.lower().replace('–','').replace('  ',' ').split(' ')
  sep='-'
  res=sep.join(productName)
  return res

In [8]:
def removeWhiteSpaces(ch):
    return " ".join(ch.split())
def getProductsInfo(productName):
  productUrl=f'https://ticka.tech/product/{productName}'
  page=requests.get(productUrl)
  if page.status_code !=200 :
    return "",""
  scoup=BeautifulSoup(page.content,"html.parser")
  infoDiv=scoup.find("ul",class_="list-unstyled")
  details=infoDiv.find_all("span",class_="details")
  rate=infoDiv.find("li",class_="clearfix")
  prefix=["Store Name : ","vendor Name : ","Adresse : "]
  for i in range(len(details)) :
    details[i]=prefix[i]+removeWhiteSpaces(details[i].text)
  rate=removeWhiteSpaces(rate.text)
  return details ,rate

In [9]:
def getItemsName(results,products):
  job_elements = results.find_all("div",class_="product-info")
  for i  in tqdm(range(len(job_elements))) :
    title_element = job_elements[i].find("h2", class_="woocommerce-loop-product__title")
    price=job_elements[i].find("span",class_="woocommerce-Price-amount amount")
    price=re.sub('[^0-9.]','',str(price))+" TND"
    title_element=preprocessProductName(title_element.text)
    info,rate=getProductsInfo(title_element)
    products.append({"product":title_element,"price":price,"info":info,"rate":rate})
  return products

In [10]:
products=getItemsName(results,[])

100%|██████████| 24/24 [01:15<00:00,  3.13s/it]


In [11]:
def getSubPageItems(products):
    for num_page in range(2,7):
        sub_page = requests.get(sub_pages_url+str(num_page))
        scoup = BeautifulSoup(sub_page.content, "html.parser")
        results = scoup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")
        products=getItemsName(results,products)
    return products
products=getSubPageItems(products)

100%|██████████| 24/24 [01:18<00:00,  3.27s/it]


In [12]:
landingPageUrl="https://ticka.tech/#"
page=requests.get(landingPageUrl)
scoup = BeautifulSoup(page.content,"html.parser")
questions=scoup.find_all("a",class_="haru-accordion-title")

In [13]:
def getTickaInformations(tickaInformation):
  landingPageUrl="https://ticka.tech/#"
  page=requests.get(landingPageUrl)
  scoup = BeautifulSoup(page.content,"html.parser")
  active="active"
  result=scoup.find_all("div",class_="haru-tab-content haru-clearfix")
  activeItem=scoup.find("div",class_="haru-tab-content haru-clearfix active")
  full_text="Question : "+questions[0].text+"\nReponse : "+activeItem.find("span").text
  tickaInformation.append({"content":full_text})
  for i in tqdm(range(len(result))):
    full_text="Question : "+questions[i+1].text+"\nReponse : "+result[i].find("span").text
    tickaInformation.append({"content":full_text})
  return tickaInformation

In [14]:
tickaInformation=getTickaInformations([])

100%|██████████| 11/11 [00:00<00:00, 28427.20it/s]


In [15]:
products.append({"Informations":tickaInformation})
prod={"storeInfo":products}

In [16]:
data=json.dumps(prod)

In [17]:
with open("data.json","w") as file : 
    file.write(data)

In [18]:
model_repo ="mistralai/Mistral-7B-Instruct-v0.2"
def tokenize():
    tokenizer=AutoTokenizer.from_pretrained(model_repo)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

def uploadModel(tokenizer):
  compute_dtype = getattr(torch, "float16")
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    )
  model=AutoModelForCausalLM.from_pretrained(model_repo,
                             quantization_config=bnb_config)
  llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.6,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
    )
  mistral = HuggingFacePipeline(pipeline=llm)
  return mistral

In [19]:
tokenizer=tokenize()
model=uploadModel(tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [20]:
template="""Instruction : [INST]You are a helpful AI system, which answers the user's question for an e-commerce startup based on the relevant data provided to you.
Answer the user's question without providing any Explanation.
If you don't know the answer, just say I don't know.
Contextual Comprehension: Unravel the meaning and tone.
Here is the data you need :
{data}
User Question :
{question}
"""
parser=StrOutputParser()
def getPrompt():
    prompt=PromptTemplate(
        input_variables=["question"],
        template=template,
    )
    return prompt
prompt=getPrompt()

In [21]:
def split_text(json_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250, chunk_overlap=30
    )
    doc=text_splitter.split_documents(json_data)
    return doc
def retiverFaiss(json_data):
    doc=split_text(json_data)
    db=FAISS.from_documents(
        doc,
        HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    )
    retiver=db.as_retriever()
    return retiver

In [22]:
def getCompletion(prompt,retiver,model,question):
    rag_chain = ({
        "data":retiver,
        "question":RunnablePassthrough()
    }|prompt
    |model
    |parser
  )
    result=rag_chain.invoke(question)
    return result


In [23]:
def loadJsonData():
    loader = JSONLoader(
        file_path='/kaggle/working/data.json',
        jq_schema='.storeInfo[]',
        text_content=False
    )
    return loader.load()

In [24]:
jsonData=loadJsonData()

In [25]:
retiver=retiverFaiss(jsonData)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
results=getCompletion(prompt,retiver,model,question="Give me 3 products from Tinbri Tounsi")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
